# CR_Score Playbook 03: Visualization & Reporting

**Level:** Intermediate  
**Time:** 15-20 minutes  
**Goal:** Create beautiful visualizations and professional reports

## What You'll Learn

- Binning visualizations (WoE, IV, distributions)
- Scorecard performance plots (ROC, KS, calibration)
- Score distribution analysis
- Generate HTML reports

## Prerequisites

- Completed Playbook 01

## Step 1: Setup and Build Scorecard

In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

from cr_score import ScorecardPipeline
from cr_score.viz import BinningVisualizer, ScoreVisualizer
from cr_score.reporting import HTMLReportGenerator

print("[OK] Libraries imported!")

[OK] Libraries imported!


In [2]:
# Load data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Build scorecard
pipeline = ScorecardPipeline(max_n_bins=5, pdo=20, base_score=600)
pipeline.fit(train_df, target_col='default')
scores = pipeline.predict(test_df)
probas = pipeline.predict_proba(test_df)

print("[OK] Scorecard built!")

{"event": "================================================================================", "timestamp": "2026-01-16T13:13:33.154359Z", "level": "info"}
{"event": "Starting CR_Score Pipeline", "timestamp": "2026-01-16T13:13:33.154359Z", "level": "info"}
{"event": "================================================================================", "timestamp": "2026-01-16T13:13:33.155358Z", "level": "info"}
{"event": "\n[1/5] Auto-binning with OptimalBinning...", "timestamp": "2026-01-16T13:13:33.155358Z", "level": "info"}
{"event": "Auto-binning 15 features", "timestamp": "2026-01-16T13:13:33.156359Z", "level": "info"}
{"max_n_bins": 5, "monotonic_trend": "auto", "event": "Fitting OptimalBinning for application_id", "timestamp": "2026-01-16T13:13:33.156359Z", "level": "info"}
{"n_bins": 4, "iv": 0.1642752080448383, "event": "OptimalBinning completed", "timestamp": "2026-01-16T13:13:33.205696Z", "level": "info"}
{"event": "  application_id: IV=0.164 - INCLUDED", "timestamp": "2026-01-1

c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


## Step 2: Binning Visualizations

In [ ]:
# Create binning visualizer
from cr_score.binning import OptBinningWrapper

# Get one feature's binning
feature = 'debt_to_income_ratio'
binner = OptBinningWrapper(max_n_bins=5)
# Note: fit() takes a Series (not DataFrame) and target Series
binner.fit(train_df[feature], train_df['default'])

# Visualize
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Get binning table
bin_table = binner.get_binning_table()

# Distribution
ax = axes[0, 0]
ax.bar(range(len(bin_table)), bin_table['count'])
ax.set_title(f'{feature}: Distribution by Bin')
ax.set_xlabel('Bin')
ax.set_ylabel('Count')

# Event Rate
ax = axes[0, 1]
ax.bar(range(len(bin_table)), bin_table['event_rate'], color='orange')
ax.set_title(f'{feature}: Default Rate by Bin')
ax.set_xlabel('Bin')
ax.set_ylabel('Default Rate')

# WoE
ax = axes[1, 0]
ax.bar(range(len(bin_table)), bin_table['woe'], color='green')
ax.set_title(f'{feature}: Weight of Evidence (WoE)')
ax.set_xlabel('Bin')
ax.set_ylabel('WoE')

# IV
ax = axes[1, 1]
ax.bar(range(len(bin_table)), bin_table['iv'], color='red')
ax.set_title(f'{feature}: Information Value (IV) by Bin')
ax.set_xlabel('Bin')
ax.set_ylabel('IV')

plt.tight_layout()
plt.show()

print(f"Total IV for {feature}: {bin_table['iv'].sum():.3f}")

TypeError: OptBinningWrapper.fit() got an unexpected keyword argument 'feature_names'

## Step 3: Scorecard Performance Plots

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
fpr, tpr, _ = roc_curve(test_df['default'], probas)
auc = roc_auc_score(test_df['default'], probas)

ax = axes[0]
ax.plot(fpr, tpr, linewidth=2, label=f'ROC (AUC={auc:.3f})')
ax.plot([0, 1], [0, 1], 'k--', label='Random')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve')
ax.legend()
ax.grid(alpha=0.3)

# Score Distribution
goods = scores[test_df['default'] == 0]
bads = scores[test_df['default'] == 1]

ax = axes[1]
ax.hist(goods, bins=30, alpha=0.6, label='Good', color='green')
ax.hist(bads, bins=30, alpha=0.6, label='Bad', color='red')
ax.set_xlabel('Credit Score')
ax.set_ylabel('Count')
ax.set_title('Score Distribution')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"[OK] Performance visualizations created!")

## Step 4: Generate HTML Report

In [ ]:
# Generate comprehensive report
report_gen = HTMLReportGenerator()

report_html = report_gen.generate(
    pipeline=pipeline,
    test_df=test_df,
    target_col='default',
    report_title="Credit Scorecard Report",
    author="Your Name"
)

# Save report
with open('scorecard_report.html', 'w', encoding='utf-8') as f:
    f.write(report_html)

print("[OK] HTML report generated: scorecard_report.html")
print("Open it in your browser to see the full report!")

## Summary

You learned how to:
- Visualize binning results (WoE, IV, distributions)
- Create performance plots (ROC, KS, calibration)
- Analyze score distributions
- Generate professional HTML reports

**Next:** Playbook 04 for the complete workflow!